In [1]:
from collections import defaultdict, Counter
import numpy as np
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications import VGG16
import cv2
import glob
import os
import shutil
from sklearn.cluster import KMeans

## Global

### Variables

In [36]:
dataset_dir = "data/traffic-small"
dataset = glob.glob(dataset_dir + "/*.jpg")

feature_saved_dir = "extracted_features"

pretrain_input_size = (64, 64)

### Utils

In [35]:
def save_result_as_file(prediction, file_name="prediction.dat"):
    r""" Save the predicted result as a new file """
    file_content = "\n".join(list(map(str, prediction)))
    with open(file_name, "w") as fd:
        fd.write(file_content) 

## Analytics
Analyze the data distribution first, so we can decide how to resize the image

In [31]:
dim_to_key = lambda dim: "({},{})".format(dim[0], dim[1])

In [32]:
cnt = Counter()

images = []
for path in dataset:
    img = cv2.imread(path)
    key = dim_to_key(img.shape)
    cnt[key] += 1

In [33]:
cnt.most_common()

[('(61,35)', 477),
 ('(61,33)', 349),
 ('(44,24)', 304),
 ('(57,27)', 300),
 ('(65,33)', 295),
 ('(67,32)', 286),
 ('(62,29)', 268),
 ('(57,29)', 263),
 ('(56,33)', 262),
 ('(72,40)', 258),
 ('(62,39)', 257),
 ('(55,32)', 252),
 ('(54,23)', 251),
 ('(49,22)', 234),
 ('(60,36)', 227),
 ('(52,25)', 224),
 ('(52,27)', 220),
 ('(55,26)', 214),
 ('(57,28)', 213),
 ('(46,27)', 211),
 ('(81,42)', 210),
 ('(63,32)', 207),
 ('(53,32)', 205),
 ('(56,24)', 205),
 ('(71,38)', 204),
 ('(70,32)', 200),
 ('(46,24)', 198),
 ('(62,34)', 196),
 ('(59,34)', 195),
 ('(77,146)', 194),
 ('(63,34)', 192),
 ('(72,38)', 190),
 ('(97,38)', 186),
 ('(64,35)', 185),
 ('(59,32)', 185),
 ('(59,26)', 185),
 ('(53,24)', 184),
 ('(63,28)', 183),
 ('(54,24)', 181),
 ('(54,26)', 178),
 ('(47,21)', 175),
 ('(57,26)', 175),
 ('(50,24)', 174),
 ('(51,27)', 172),
 ('(99,44)', 171),
 ('(54,33)', 169),
 ('(57,31)', 169),
 ('(59,31)', 168),
 ('(64,33)', 167),
 ('(57,30)', 162),
 ('(48,21)', 162),
 ('(56,30)', 161),
 ('(44,26)'

## Resize

In [53]:
images = []
for path in dataset:
    img = cv2.imread(path)
    img = cv2.resize(img, pretrain_input_size)
    img = img / 255.0  # Normalization
    images.append(img)

In [54]:
train_input = np.array(images)

## Feature Extraction 

In [70]:
def init_feature_saved_dir():
    try:
        shutil.rmtree(feature_saved_dir)
    except:
        pass

    os.mkdir(feature_saved_dir)

In [71]:
def save_batch_features(x):
    init_feature_saved_dir()

    vgg16 = VGG16(weights='imagenet', include_top=False)
    y = vgg16.predict(x)
    
    np.save("%s/features" % feature_saved_dir, y)

In [72]:
save_batch_features(train_input)

## Clustering

In [3]:
model = KMeans(n_clusters=14)

In [7]:
x = np.load(os.path.join(feature_saved_dir, "features.npy"))

In [27]:
x_flatten = np.array([np.ndarray.flatten(img) for img in x])

In [28]:
x_flatten.shape

(4209, 2048)

In [29]:
model.fit(x_flatten)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=14, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [37]:
labels = model.labels_.tolist()

In [38]:
save_result_as_file(labels, "kmeans_1st_try.dat")